QC of ETL starting with GDC release 22 for programs TARGET, ORGANOID, and BEATAML. 


This notebook focuses on the QC of program TARGET data_type miRNA-Expression.

##QC table checklist 

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

**2. Look at table row number and size**

Do these metrics make sense?

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

**4. Number of cases on GDC portal versus table?**

**5. Number of cases / aliquots versus BigQuery metadata table**

**6. Number of entries per miRNA_id - should equal aliquot count**

**7.Check for any duplicate rows present in the table**

##Reference material



*   [NextGenETL](https://github.com/isb-cgc/NextGenETL) GitHub repository
*   [ETL QC SOP draft](https://docs.google.com/document/d/1Wskf3BxJLkMjhIXD62B6_TG9h5KRcSp8jSAGqcCP1lQ/edit)

##Before you begin

You need to load the BigQuery module, authenticate ourselves, create a client variable, and load the necessary libraries.


In [1]:
from google.colab import auth
try:
  auth.authenticate_user()
  print('You have been successfully authenticated!')
except:
  print('You have not been authenticated.')

You have been successfully authenticated!


In [2]:
from google.cloud import bigquery
try:
  project_id = 'isb-project-zero' # Update your_project_number with your project number
  client = bigquery.Client(project=project_id)
  print('BigQuery client successfully initialized')
except:
  print('Failed')

BigQuery client successfully initialized


In [3]:
#Install pypika to build a Query 
!pip install pypika
# Import from PyPika
from pypika import Query, Table, Field, Order

import pandas

     |████████████████████████████████| 61kB 2.8MB/s 
  Created wheel for pypika: filename=PyPika-0.37.8-py2.py3-none-any.whl size=42761 sha256=e76e7f1f8bf7d9fc2ffc60907270b41b6ead6b831df6a1455b88c06c2f5b000c
  Stored in directory: /root/.cache/pip/wheels/98/e5/b1/34dff2bf771d27d2bac48984e1855dee810721f09a8d7d2e7d
Successfully built pypika


## READY TO BEGIN TESTING

##Program TARGET data_type: miRNA Expression

**Testing Full ID** `isb-project-zero.fs_scratch.TARGET_HG38_miRNA_Expression_20200430`

[Table location](https://console.cloud.google.com/bigquery?authuser=2&project=isb-project-zero&p=isb-project-zero&d=fs_scratch&t=TARGET_HG38_miRNA_Expression_20200430&page=table)

Source : GDC API

Release version : v22


##test 1 - schema verification

**1. Check schema**

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields
    
Are the labels correct

Google documentation column descriptions for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#column_field_paths_view).

Google documentation table options for [reference](https://cloud.google.com/bigquery/docs/information-schema-tables#options_table).

In [ ]:
#return all table information for dataset TARGET_HG38_miRNA_Expression_20200430 

mirnaseq_table = Table('`isb-project-zero`.fs_scratch.INFORMATION_SCHEMA.TABLES')
mirnaseq_query = Query.from_(mirnaseq_table) \
                  .select(' table_catalog, table_schema, table_name, table_type ') \
                  .where(mirnaseq_table.table_name=='TARGET_HG38_miRNA_Expression_20200430') \
                  
mirnaseq_query_clean = str(mirnaseq_query).replace('"', "")
mirnaseq = client.query(mirnaseq_query_clean).to_dataframe()
mirnaseq.head()

,table_catalog,table_schema,table_name,table_type
0,isb-project-zero,fs_scratch,TARGET_HG38_miRNA_Expression_20200430,BASE TABLE


In [ ]:
#return all table information for dataset TARGET_HG38_miRNA_Expression_20200430 
mirnaseq_table = Table('`isb-project-zero`.fs_scratch.INFORMATION_SCHEMA.TABLE_OPTIONS')
mirnaseq_query = Query.from_(mirnaseq_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(mirnaseq_table.table_name=='TARGET_HG38_miRNA_Expression_20200430') \

mirnaseq_query_clean = str(mirnaseq_query).replace('"', "")
mirnaseq = client.query(mirnaseq_query_clean).to_dataframe()
pandas.options.display.max_rows


for i in range(len(mirnaseq)):
    print(mirnaseq['option_name'][i] + '\n')
    print('\t' + mirnaseq['option_value'][i] + '\n')
    print('\t' + mirnaseq['option_type'][i] + '\n')

friendly_name

	"TARGET HG38 MIRNASEQ"

	STRING

description

	"Data was extracted from the release 22 active archive of the GDC in June 2020 microRNA expression data for TARGET samples."

	STRING

labels

	[STRUCT("access", "open"), STRUCT("data_type", "mirna_expression"), STRUCT("program", "target"), STRUCT("reference_genome_0", "hg38"), STRUCT("source", "gdc"), STRUCT("category", "processed_-omics_data"), STRUCT("status", "current")]

	ARRAY<STRUCT<STRING, STRING>>



In [ ]:
#check for empty schemas in dataset TARGET_HG38_miRNA_Expression_20200430 

mirnaseq_table = Table('`isb-project-zero`.fs_scratch.INFORMATION_SCHEMA.TABLE_OPTIONS')
mirnaseq_query = Query.from_(mirnaseq_table) \
                  .select(' table_name, option_name, option_type, option_value ') \
                  .where(mirnaseq_table.table_name=='TARGET_HG38_miRNA_Expression_20200430') \

mirnaseq_query_clean = str(mirnaseq_query).replace('"', "")
mirnaseq = client.query(mirnaseq_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
mirnaseq.empty

Are there any empty cells in the table schema?


False

FIELD Descriptions pulled example below


In [ ]:
#list of field descriptions for table 

#return all table information for dataset TARGET_HG38_miRNA_Expression_20200430

mirnaseq_table = Table('`isb-project-zero`.fs_scratch.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
mirnaseq_query = Query.from_(mirnaseq_table) \
                  .select('table_name, column_name, description') \
                  .where(mirnaseq_table.table_name=='TARGET_HG38_miRNA_Expression_20200430') \

mirnaseq_query_clean = str(mirnaseq_query).replace('"', "")
mirnaseq = client.query(mirnaseq_query_clean).to_dataframe()
pandas.options.display.max_rows

for i in range(len(mirnaseq)):
  print(mirnaseq['table_name'][i] + '\n')
  print('\t' + mirnaseq['column_name'][i] + '\n')
  print('\t' + mirnaseq['description'][i] + '\n')

TARGET_HG38_miRNA_Expression_20200430

	project_short_name

	Project name abbreviation, eg TARGET-AML

TARGET_HG38_miRNA_Expression_20200430

	case_barcode

	Original TARGET case barcode, eg TARGET-20-PASCGR

TARGET_HG38_miRNA_Expression_20200430

	sample_barcode

	TARGET sample barcode, eg TARGET-20-PASWAT-09A

TARGET_HG38_miRNA_Expression_20200430

	aliquot_barcode

	TARGET aliquot barcode, eg TARGET-20-PAJLIP-01A-01R

TARGET_HG38_miRNA_Expression_20200430

	miRNA_id

	Unique miRNA id (aka symbol), eg hsa-mir-21  --  relevant reference information can be found in the isb-cgc:genome_reference dataset in the tables miRBase_v21 and miRBase_v21_hsa_gff3

TARGET_HG38_miRNA_Expression_20200430

	read_count

	Number of reads that were mapped to this miRNA

TARGET_HG38_miRNA_Expression_20200430

	reads_per_million_miRNA_mapped

	Read count normalized by total reads mapped divided by 1 million

TARGET_HG38_miRNA_Expression_20200430

	cross_mapped

	A short isomiR read may map exactly to matur

In [ ]:
#list of field descriptions for table 

#check for empty schemas in dataset TARGET_HG38_miRNA_Expression_20200430 
mirnaseq_table = Table('`isb-project-zero`.fs_scratch.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS')
mirnaseq_query = Query.from_(mirnaseq_table) \
                  .select('table_name, column_name, description') \
                  .where(mirnaseq_table.table_name=='TARGET_HG38_miRNA_Expression_20200430') \

mirnaseq_query_clean = str(mirnaseq_query).replace('"', "")
mirnaseq = client.query(mirnaseq_query_clean).to_dataframe()
pandas.options.display.max_rows
print("Are there any empty cells in the table schema?")
mirnaseq.empty

Are there any empty cells in the table schema?


False

##test 2 row number verification

**2. Look at table row number and size**

Do these metrics make sense?

In [ ]:
%%bigquery --project isb-project-zero
SELECT COUNT(case_gdc_id)
FROM `isb-project-zero.fs_scratch.TARGET_HG38_miRNA_Expression_20200430`

,f0_
0,4473018


In [ ]:
%%bigquery --project isb-project-zero
SELECT *
FROM `isb-project-zero.fs_scratch.TARGET_HG38_miRNA_Expression_20200430`
limit 5

,project_short_name,case_barcode,sample_barcode,aliquot_barcode,miRNA_id,read_count,reads_per_million_miRNA_mapped,cross_mapped,file_gdc_id,case_gdc_id,sample_gdc_id,aliquot_gdc_id
0,TARGET-RT,TARGET-52-PADYCE,TARGET-52-PADYCE-01A,TARGET-52-PADYCE-01A-01R,hsa-mir-125b-1,15446,1754.648637,N,f1203cbf-c8f4-447f-931b-e19b0c4f234f,e1e96595-1e65-5548-a40a-b7a7332a846b,45cbf2f5-5ba7-5e36-9085-a476eda0396f,edb656e1-0250-52ae-99d0-2de81a98ccfd
1,TARGET-RT,TARGET-52-PASGCL,TARGET-52-PASGCL-01A,TARGET-52-PASGCL-01A-02R,hsa-mir-4461,21,4.684184,N,30fe4bdc-ba57-4c50-bc36-7a54dce03a7b,6730faef-1a6a-58dc-96e9-e260b51e5770,83acde99-6105-516e-b0c2-23059e62462a,69255862-ba6a-537b-9242-49d36df3131c
2,TARGET-RT,TARGET-52-PAKLYZ,TARGET-52-PAKLYZ-01A,TARGET-52-PAKLYZ-01A-01R,hsa-mir-24-1,14292,1471.249639,N,5a09d94c-617d-43f0-9402-3aabe2be5584,97b2dd12-5737-5c2b-9088-1915fec43003,6fb25ec4-efd2-5b20-b03f-f223168f6093,3c6d3e87-f189-570b-908d-15332c19a0d7
3,TARGET-RT,TARGET-52-PAREWI,TARGET-52-PAREWI-01A,TARGET-52-PAREWI-01A-02R,hsa-mir-29b-1,1255,335.043434,N,f10fb0f4-d6ca-4354-9d44-b8d013f71022,16815c4e-20e2-5b5d-80ef-01971b854665,734f9185-68f2-5432-810f-e872813a2ded,7b4e1f26-760b-5273-8b91-770d33010156
4,TARGET-RT,TARGET-52-PATXEE,TARGET-52-PATXEE-01A,TARGET-52-PATXEE-01A-01R,hsa-mir-664b,12,1.922506,N,9f001ff1-9d47-450f-b288-d20e99030dd1,7b97d1d3-acf1-5be5-bb8c-9721b355ec71,7d0f955a-386c-56f2-ab0b-080c6f330ae6,91933210-d7ec-5b56-8e9a-c8098ed07b71


##test 3 - manual verification

**3. Scroll through table manually**

See if anything stands out - empty columns, etc.

The BigQuery table search user interface is useful in for this test run. The test tier points to the isb-etl-open. 

ISB-CGC BigQuery table  search [test tier](https://isb-cgc-test.appspot.com/bq_meta_search/).

BigQuery console [isb-project-zero](https://console.cloud.google.com/bigquery?authuser=1&folder=&organizationId=&project=isb-project-zero&p=isb-project-zero&d=fs_scratch&t=TARGET_HG38_miRNA_Expression_20200430&page=table).

Run a manual check in the console with the steps mentioned in step 1 

Are all the fields labeled?

Is there a table description?

Do the field labels make sense for all fields?
    
Are the labels correct?

##test 4 - GDC Data Portal count verfication


**4. Number of cases on GDC portal versus table?**

In [ ]:
# Query below will display the number of cases presents in this table.

mirnaseq_table = Table('`isb-project-zero.fs_scratch.TARGET_HG38_miRNA_Expression_20200430`')
mirnaseq_query = Query.from_(mirnaseq_table) \
                  .select(' DISTINCT case_barcode, count(*) as count') \
                  .groupby('case_barcode')

mirnaseq_query_clean = str(mirnaseq_query).replace('"', "")
#print(mirnaseq_query_clean)
mirnaseq = client.query(mirnaseq_query_clean).to_dataframe()
print('number of cases = ' + str(len(mirnaseq.index)))


number of cases = 2258


To compare against the GDC Data Portal, 
you first go the Data Portal and search for program TARGET and experimental_strategy miRNA-Seq, the cases number returned is 2,258. 

[GDC Data portal](https://portal.gdc.cancer.gov/repository?facetTab=files&filters=%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22cases.project.program.name%22%2C%22value%22%3A%5B%22TARGET%22%5D%7D%7D%2C%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22files.experimental_strategy%22%2C%22value%22%3A%5B%22miRNA-Seq%22%5D%7D%7D%5D%7D&searchTableTab=cases) filter results. 

##test 5 - file metadata table count verification

**5. Number of cases / aliquots versus BigQuery metadata table**

In [ ]:
# Query below will display the number of cases presents in this table.

mirnaseq_table = Table('`isb-project-zero.fs_scratch.TARGET_HG38_miRNA_Expression_20200430`')
mirnaseq_query = Query.from_(mirnaseq_table) \
                  .select(' DISTINCT case_barcode, count(*) as count') \
                  .groupby('case_barcode')

mirnaseq_query_clean = str(mirnaseq_query).replace('"', "")
#print(mirnaseq_query_clean)
mirnaseq = client.query(mirnaseq_query_clean).to_dataframe()
print('number of cases = ' + str(len(mirnaseq.index)))


number of cases = 2258


In [ ]:
# cases count from gdc metadata table 

%%bigquery --project isb-project-zero
SELECT case_gdc_id, program_name
FROM `isb-project-zero.GDC_metadata.rel23_fileData_current`
where program_name = 'TARGET'
and experimental_strategy = 'miRNA-Seq'
group by case_gdc_id, program_name

,case_gdc_id,program_name
0,214e8f3a-0229-583b-935d-a5c64b3aa58e,TARGET
1,9e265b0d-4497-5824-880d-0a0c80c1dfd8,TARGET
2,4bb21e43-d465-5f6a-96a4-c8682ae3faf8,TARGET
3,0b372c9b-2d5a-5483-8c96-7f829d962a99,TARGET
4,6ed0b780-1b87-54ce-a036-8e74ece2a705,TARGET
...,...,...
2253,ddb89d20-f1a2-4e07-ac59-151d9d3ed446,TARGET
2254,d45e8061-54b5-4702-b0df-4592aa36d558,TARGET
2255,e98b3034-9fa7-43c5-8e25-9321b4dccdec,TARGET
2256,b99603f0-f12c-454e-8a24-75543e717cba,TARGET


In [6]:
# Aliquot counts table results

mirnaseq_table = Table('`isb-project-zero.fs_scratch.TARGET_HG38_miRNA_Expression_20200430`')
mirnaseq_query = Query.from_(mirnaseq_table) \
                  .select(' distinct aliquot_gdc_id, count(*) as count') \
                  .groupby('aliquot_gdc_id')

mirnaseq_query_clean = str(mirnaseq_query).replace('"', "")

mirnaseq = client.query(mirnaseq_query_clean).to_dataframe()
print('number of aliquots = ' + str(len(mirnaseq.index)))


number of aliquots = 2378


In [ ]:
# GDC file metadata table aliquot count 

%%bigquery --project isb-project-zero
select distinct associated_entities__entity_gdc_id 
from `isb-cgc.GDC_metadata.rel22_fileData_active` 
where program_name = "TARGET"
and experimental_strategy = "miRNA-Seq"

,associated_entities__entity_gdc_id
0,830a99ef-759e-528e-a30d-01530b376ba5
1,007a5a35-5614-52d3-8393-7642ecf84933
2,99c936d5-9c78-529f-b8bc-812176fd7979
3,50a7414b-8002-51d4-ac29-93721c8627e0
4,12415d0f-1d39-581a-acb6-df908c6ce94d
...,...
2373,fdaabad1-2136-42e9-b656-0dd444199a18
2374,fc91cae7-467d-45cf-973d-fb8c7274174c
2375,fff263a6-525c-476c-9900-eba12a7df0f8
2376,fedd709d-f830-4f2e-bdcb-e411a9172508


## test 6 - miRNA_id entry verification

**6. Number of entries per miRNA_id - should equal aliquot count**

In [ ]:
%%bigquery --project isb-project-zero

select distinct miRNA_id, count(miRNA_id) as count
from `isb-project-zero.fs_scratch.TARGET_HG38_miRNA_Expression_20200430` 
group by miRNA_id 
order by count

,miRNA_id,count
0,hsa-mir-125b-1,2378
1,hsa-mir-4461,2378
2,hsa-mir-24-1,2378
3,hsa-mir-29b-1,2378
4,hsa-mir-664b,2378
...,...,...
1876,hsa-mir-3178,2378
1877,hsa-mir-6847,2378
1878,hsa-mir-1245b,2378
1879,hsa-mir-4301,2378


##step 7 - duplication verifcation

**7.Check for any duplicate rows present in the table**



In [ ]:
%%bigquery --project isb-project-zero

SELECT DISTINCT case_barcode, count(*) as count
from `isb-project-zero.fs_scratch.TARGET_HG38_miRNA_Expression_20200430`
group by project_short_name, case_barcode, sample_barcode, aliquot_barcode, miRNA_id, read_count, reads_per_million_miRNA_mapped, cross_mapped, file_gdc_id, case_gdc_id, sample_gdc_id, aliquot_gdc_id
order by count desc
limit 10

,case_barcode,count
0,TARGET-20-PAVNHN,1
1,TARGET-20-PAXBLW,1
2,TARGET-20-PANTPW,1
3,TARGET-20-PARXJW,1
4,TARGET-20-PATAVF,1
5,TARGET-20-PAXARD,1
6,TARGET-20-PAUXYG,1
7,TARGET-20-PANUUA,1
8,TARGET-20-PARIYB,1
9,TARGET-20-PAUNSV,1
